#Química das/nas Nuvens

A química da atmosfera oferece novos pontos de vista sobre questões importantes nos dias de hoje, tais como o **aquecimento global**. Este _notebook_ oferece uma forma lúdica e interativa de compreender os fenómenos por detrás do efeito de estufa e de como diferentes tipos de emissões têm um impacto diferenciado no aquecimento global. No final do dia, são apenas moléculas a fazer o seu dia-a-dia, vibrando alegremente na atmosfera ao sabor dos fotões que recebem do Sol. Cabe a nós reflectir sobre o seu impacto nas nossas vidas!

## Instruções Básicas

Os notebooks do colab oferecem uma forma fácil de correr código Python devidamente anotado. Cada notebokk está divido em células, que podem conter texto (anotações) ou código para executar. Desta forma, a exploração dos conteúdos pode ser feita de forma iterativa, bastando saber algumas regras básicas de manipulação:
1. Cada célula é executada clicando no símbolo de "play" no canto superior esquerdo, ou com a combinação `Shift + Enter`.
1. Algumas células com código demoram tempo a correr, enquanto a célula está a correr, aparece um ícone animado no canto superior esquerdo da mesma.
1. se colocarmos várias células a correr, estas são executadas por ordem (uma de cada vez).
1. Ao longo do notebook, aparecem sugestões de alterações que podem ser feitas para explorar outros aspectos.

### Preparação do _notebook_

As duas células abaixo devem ser executadas na primeira vez que abrimos o _notebook_ de forma a preparar o ambiente de trabalho. Após isso, pode clicar na seta para baixo (junto ao título "Preparação do _notebook_" para compactar esta secção e passar para a secção seguinte.

In [2]:
%%capture
%%bash

pip install py3Dmol
pip install seaborn

# install XTB
wget -c https://github.com/grimme-lab/xtb/releases/download/v6.5.1/xtb-6.5.1-linux-x86_64.tar.xz
tar xf xtb-6.5.1-linux-x86_64.tar.xz
export PATH=/content/xtb-6.5.1/bin:${PATH}
# get data
wget -c https://raw.githubusercontent.com/teixeirafilipe/ChemOutReach/main/pt/NEI_2022/solar_data.csv


In [136]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import os

import pandas as pd
import numpy as np
import time
from collections import defaultdict
#IPythonConsole.ipython_3d = True

import py3Dmol
import matplotlib.pyplot as plt
import subprocess
import seaborn as sb

mol_d=dict()
mol_d['H2O']=dict()
mol_d['H2O']['name']='Água'
mol_d['H2O']['geom']=""" O     0.000000     0.000000     0.000000
 H     0.000000     0.000000     0.950000
 H     0.895670     0.000000    -0.316663
"""

mol_d['CO']=dict()
mol_d['CO']['name']='Monóxido de Carbono'
mol_d['CO']['geom']=""" C 0.0 0.0 0.0
 O 0.0 0.0 1.2
"""

mol_d['CO2']=dict()
mol_d['CO2']['name']='Dióxido de Carbono'
mol_d['CO2']['geom']=""" C 0.0 0.0 0.0
 O 0.0 0.0  1.2
 O 0.0 0.0 -1.2
"""

mol_d['CH4']=dict()
mol_d['CH4']['name']='Metano'
mol_d['CH4']['geom']=""" C     0.000000     0.000000     0.000000
 H     0.000000     0.000000     1.089000
 H     1.026720     0.000000    -0.362996
 H    -0.513360    -0.889165    -0.363000
 H    -0.513360     0.889165    -0.363000
"""

mol_d['H2S']=dict()
mol_d['H2S']['name']='Sulfureto de Hidrogéio'
mol_d['H2S']['geom']=""" S     0.000000     0.000000     0.000000
 H     0.000000     0.000000     0.950000
 H     0.895670     0.000000    -0.316663
"""

mol_d['NH3']=dict()
mol_d['NH3']['name']='Amoníaco'
mol_d['NH3']['geom']=""" N     0.000000     0.000000     0.000000
 H     1.026720     0.000000    -0.362996
 H    -0.513360    -0.889165    -0.363000
 H    -0.513360     0.889165    -0.363000
"""

mol_d['SO2']=dict()
mol_d['SO2']['name']='Dióxido de Enxofre'
mol_d['SO2']['geom']=""" S     0.000000     0.000000     0.000000
 O     0.000000     0.000000     1.510000
 O     1.307698     0.000000    -0.755000
"""

mol_d['SO3']=dict()
mol_d['SO3']['name']='Trióxido de Enxofre'
mol_d['SO3']['geom']=""" S     0.000000     0.000000     0.000000
 O     0.000000     0.000000     1.510000
 O     1.307698     0.000000    -0.755000
 O    -1.307698     0.000000    -0.755000
"""

mol_d['SF6']=dict()
mol_d['SF6']['name']='Hexafluoreto de Enxofre'
mol_d['SF6']['geom']=""" S     0.000000     0.000000     0.000000
 F     0.000000     0.000000     1.670000
 F     1.670000     0.000000     0.000000
 F     0.000000    -1.670000     0.000000
 F    -1.670000     0.000000     0.000000
 F     0.000000     1.280000     0.000000
 F     0.000000     0.000000    -1.670000
"""

atomic_symbols=['XX','H','He','Li','Be','B','C','N','O','F','Ne',
                'Na','Mg','Al','Si','P','S','Cl','Ar']


def shell(cmd, shell=True):
  "runs a command in the linux shell, adapted from Jimmy Kromann."
  if shell:
    p = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  else:
    p = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  output, err = p.communicate()
  return output

def run_xtb(args):
  return shell(f"ulimit -s unlimited; OMP_STACKSIZE=12G OMP_MAX_ACTIVE_LEVELS=1 /content/xtb-6.5.1/bin/xtb {args}")

def g98_animate_mode(fn,n_vib,scale=1.0):
  data = open(fn,'r').readlines()
  ZZ = list()
  geo = list()
  displ = list()
  # ler a geometria
  geo_start=100000000000000
  geo_end = -1
  for n,line in enumerate(data):
    if 'Coordinates' in line:
      geo_start = n+3
    if (n>geo_start) and '--------' in line:
      geo_end = n
      break
  for n in range(geo_start,geo_end):
    l =  data[n].split()
    ZZ.append(int(l[1]))
    geo.append(l[-3:])
  symbols = [atomic_symbols[z] for z in ZZ]
  natoms = len(symbols)
  vib_start = geo_end + 7
  vib_step = 9 + natoms
  freq=0.0
  for i in range(vib_start,len(data),vib_step):
    if n_vib in list(map(int,data[i].split())):
      v_ind = list(map(int,data[i].split())).index(n_vib)
      v_start = i + 9
      v_end = i + vib_step
      freq = float(data[i+2].split()[2+v_ind])
      for j in range(v_start,v_end):
        l = data[j].split()
        displ.append(list(map(float,l[2+(3*v_ind):5+(3*v_ind)])))
      break
  #print(displ)
  #print(freq)
  for n in range(natoms):
    for j in range(3):
      displ[n][j] *= scale
  xyz=f"{natoms}\n* (null), Energy   -1000.0000000\n"
  for n in range(natoms):
    xyz += f"{symbols[n]}"
    for j in range(3):
      xyz += f" {geo[n][j]}"
    for j in range(3):
      xyz += f" {displ[n][j]}"
    xyz += '\n'
  xyzview = py3Dmol.view(width=250,height=250)
  xyzview.addModel(xyz,'xyz',{'vibrate': {'frames':10,'amplitude':1}})
  xyzview.setStyle({'stick':{},'sphere':{'scale':0.3}})
  xyzview.setBackgroundColor('0xeeeeee')
  #xyzview.addLabel(f"{freq:0.1f} cm⁻¹\n{10000000/freq:0.1f} nm")
  xyzview.addLabel(f"{10000000/freq:0.0f} nm")
  xyzview.animate({'loop': 'backAndForth'})
  xyzview.zoomTo()
  xyzview.show()

def plot_solar_spectrum(cdo_min= 280, cdo_max= 3000, ax=None):
  standalone=False
  if not ax:
    fig, ax = plt.subplots(figsize=(10,7.5))
    standalone=True
  raw_data = pd.read_csv('/content/solar_data.csv')
  data = raw_data.loc[(raw_data['cdo_nm']> cdo_min) & (raw_data['cdo_nm']< cdo_max)]
  ax.plot(data['cdo_nm'], data['Em_solar_Wm2nm1'], label="Espectro Solar")
  ax.plot(data['cdo_nm'], data['Em_surfaceW2m2nm1'], label="Espectro De Emissão da Terra")
  if standalone:
    ax.legend()
    ax.set_ylabel("Emisividade (W.m⁻².nm⁻¹)")
    ax.set_xlabel("Comprimento de onda (nm)")
    plt.show()
  else:
    return max(data['Em_solar_Wm2nm1'].max(),data['Em_surfaceW2m2nm1'].max())

def inspecionar(molecula, animar=True):
  # definir ficheiro da geometria
  if molecula not in mol_d:
    raise NotImplementedError(f"A molécula {molecula} não se encontra definida.")
  mol_name = mol_d[molecula]['name']
  geo_i = mol_d[molecula]['geom']
  natoms = len(geo_i.split('\n'))-1
  with open("mol_guess.xyz",'w') as f:
    f.write(f"""{natoms}
{molecula}
{geo_i}""")
  # correr xtb
  run_xtb("mol_guess.xyz --ohess --namespace mol_ohess > mol_ohess.out")
  # extrair dados do espectro de IR
  o_data = open('mol_ohess.out','r').readlines()
  linear = False
  for line in o_data:
    if ':  linear?' in line and 'false' not in line:
      linear = True
      break
  n_freqs = 3*natoms-5 if linear else 3*natoms-6
  #print(n_freqs)
  # fazer o grafico
  freqs = []
  ints = []
  v_data = open('mol_ohess.vibspectrum','r').readlines()
  for line in v_data[-(n_freqs+1):-1]:
    l=line.split()
    freqs.append(float(l[-4]))
    ints.append(float(l[-3]))
  cdo=(10000000/np.array(freqs)) #converter para comprimento de onda
  #print(freqs)
  #print(cdo)
  cdo_low = 1500
  cdo_high = 4000
  npoints= 700
  x = np.linspace(cdo_low, cdo_high, npoints)
  y = np.zeros(npoints)
  for f,i in zip(cdo,ints):
    y += i*np.exp((-(x-f)**2)/10000)
    #print(f,i)
  fig, ax= plt.subplots(figsize=(10,7.5))
  top=plot_solar_spectrum(cdo_low, cdo_high, ax=ax)
  ax.plot(x,top*(y/y.max()), label=f"{mol_name}")
  ax.set_ylabel("Emisividade/Absortividade (W.m⁻².nm⁻¹)")
  ax.set_xlabel("Comprimento de onda (nm)")
  ax.legend()
  plt.show()
  # mostrar as vibrações
  if animar:
    for i in range(n_freqs):
      if cdo[i]>=cdo_low and cdo[i]<=cdo_high:
        g98_animate_mode('mol_ohess.g98.out',i+1)

# Espectro de Emissão do Sol e da Terra

O espectro de emissão do Sol é semelhante ao espectro de emissão de um corpo negro, com um máximo de emissão a cerca de 500 nm (Figura abaixo). Uma parte significativa da emissão solar ocorre no infra-vermelho, a comprimentos de onda superiores a 1000 nm.

In [ ]:
plot_solar_spectrum()

Quando medimos a emissão do Sol reflectido pela Terra notamos a aussência de algumas bandas. Estas aussências são causadas pela absorção da radiação solar por certos elementos da atmosfera, tais como o vapor de água.

A absorção de radiação no infravermelho deve-se à vibração dos átomos que constituem as moléculas, pelo que moléculas diferentes vão absorver radiação a comprimentos de onda diferentes. Quando as moléculas absorvem fotões infravermelhos, ficam mais agitadas e notamos um aumento da temperatura. Portanto, toda a radiação absorvida acaba por contribuir para um aumento da temperatura. As moléculas estão sempre a vibrar, a emitir e a absorver fotões no infravermelho.

No entanto, há regiões onde a Terra reflete quase tanto quanto recebe. Estas regiões são como "janelas" numa estufa, pois permitem que a radiação infravermelha vá escapando para o espaço.

## Influência da Água

Para avaliar a influencia da água na radiação devolvida pela Terra ao espaço, podemos calcular o seu espectro no infravermelho. 

A célula abaixo trata de fazer esse cálculo na *cloud* da Google, e mostra o espectro de absorção da água no infravermelho sobreposto aos espectros de emissão solar e reflexão da Terra. Para além disso, também mostra animações da forma como a água vibra quando absorve a radiação indicada no espectro.


In [ ]:
inspecionar('H2O')

Como podemos ver, a molécula de água vibra de duas formas diferentes a cerca de 2740 nmm e estas vibrações são responsáveis pela aussência de radiação emitida pela Terra entre os 2500 e os 3000 nm .

# Ação do Dióxido de Carbono

Na célula abaixo, podemos inspecionar o efeito do $CO_2$ no espectro de emissão da terra.

In [ ]:
inspecionar('CO2')

O dióxido de carbono tem uma banda de absorção a cerca de 3859 nm. Relativamente ao resto do espectro, esta é uma zona onde a energia emitida e recebida não é muita, no entanto esta é uma das maiores "janelas" que a Terra tem para deixar escapar o calor!

# Ação do Metano

O Metano ($CH_4$) é um componente do gás natural, mas também é uma emissão resultante da agro-pecuária. Na célula abaixo podemos ver o seu impacto no efeito de estufa.

In [ ]:
inspecionar('CH4')

# New Section

O metano tem várias formas de vibrar que envolvem a absorção de radiação entre os 3000 e os 3500 nm. Junto com o dióxido de carbono, o metano ajuda a fechar completamente a "janela" que temos para deixar escapar o calor que a água absorve entre os 2500 e os 3000 nm!

# Outros Gases

Nas pŕoximas células podes experimentar ver o impacto de outros gases no efeito de estufa. Clica no 

In [ ]:
inspecionar('CO')

In [ ]:
inspecionar('NH3')

In [ ]:
inspecionar('H2S')